## Identifying Duplicate Questions

Over 100 million people visit Quora every month, so it's no surprise that many people ask similar (or the same) questions. Various questions with the same intent can cause people to spend extra time searching for the best answer to their question, and results in members answering multiple versions of the same question. Quora uses random forest to identify duplicated questions to provide a better experience to active seekers and writers, and offer more value to both of these groups in the long term.
Follow the steps outlined below to build the appropriate classifier model. 


Steps:
- Download data
- Exploration
- Cleaning
- Feature Engineering
- Modeling

By the end of this project you should have **a presentation that describes the model you built** and its **performance**. 


In [59]:
import pandas as pd

Link for the data<br/>
`https://drive.google.com/file/d/19iWVGLBi7edqybybam56bt2Zy7vpf1Xc/view?usp=sharing`

In [60]:
df = pd.read_csv("./data/train.csv")

#### Note
There is no designated test.csv file. The train.csv file is the entire dataset. Part of the data in the train.csv file should be set aside to act as the final testing data.

In [61]:
from sklearn.model_selection import train_test_split

In [62]:
train, test = train_test_split(df, test_size=0.2)

### Exploration

In [63]:
print("Shape of the data: ", train.shape)
print(df.dtypes)

Shape of the data:  (323432, 6)
id               int64
qid1             int64
qid2             int64
question1       object
question2       object
is_duplicate     int64
dtype: object


In [64]:
train.sample(5)

,id,qid1,qid2,question1,question2,is_duplicate
241057,241057,9769,353008,Which is the best speech you ever heard?,What are some of the best speeches you have he...,1
138612,138612,214731,220679,Did Joseph Goebbels read many books?,Was Joseph Goebbels an avid reader?,1
139988,139988,21863,222535,How do I tell her I love her?,How do I tell her I don't love her the way she...,0
29069,29069,39192,1296,What happens if you burn 1000 calories a day?,Is it possible to burn 1000 calories each day?...,0
51205,51205,90866,90867,How much Oxygen does an average person need in...,How many hairs does the average person lose pe...,0


In [65]:
train.describe()

,id,qid1,qid2,is_duplicate
count,323432.000000,323432.000000,323432.000000,323432.000000
mean,201901.858684,216926.058362,220689.600911,0.369583
std,116707.119368,157660.428282,159743.675875,0.482693
min,0.000000,1.000000,2.000000,0.000000
25%,100755.750000,74188.500000,74605.000000,0.000000
50%,201786.000000,191692.000000,196778.000000,0.000000
75%,302920.250000,346065.000000,353930.750000,1.000000
max,404288.000000,537930.000000,537931.000000,1.000000


In [66]:
train.isnull().sum()

id              0
qid1            0
qid2            0
question1       1
question2       2
is_duplicate    0
dtype: int64

In [67]:
# remove null values
train = train.dropna()

In [68]:
train.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [69]:
train.shape

(323429, 6)

### Cleaning

- Tokenization
- Stopwords cleaning
- Removing punctuation
- Normalizing
- Stemming

#### Tokenization

In [70]:
# tokenization
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\Verma
[nltk_data]     Ashiyaan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [71]:
train['tokens1'] = train['question1'].apply(word_tokenize)
train['tokens2'] = train['question2'].apply(word_tokenize)

train.head()

,id,qid1,qid2,question1,question2,is_duplicate,tokens1,tokens2
298900,298900,132471,44677,What are some lesser known facts about PM Nare...,What are some lesser known facts/incidents rel...,1,"[What, are, some, lesser, known, facts, about,...","[What, are, some, lesser, known, facts/inciden..."
101435,101435,168039,168040,How do I use PC's internet connection on Andro...,Can I use net on my Android phone from cable b...,0,"[How, do, I, use, PC, 's, internet, connection...","[Can, I, use, net, on, my, Android, phone, fro..."
349253,349253,226755,5914,How do you delete your Yahoo email account?,How can you delete your Yahoo mail account?,1,"[How, do, you, delete, your, Yahoo, email, acc...","[How, can, you, delete, your, Yahoo, mail, acc..."
68381,68381,118184,118185,What is the real world?,What is out there in the real world?,0,"[What, is, the, real, world, ?]","[What, is, out, there, in, the, real, world, ?]"
102409,102409,169441,169442,Why do artists/musicians/actors etc tend to be...,What is the future of left wing politics in In...,0,"[Why, do, artists/musicians/actors, etc, tend,...","[What, is, the, future, of, left, wing, politi..."


#### Stopwords cleaning

In [72]:
# stopwords cleaning
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to C:\Users\Verma
[nltk_data]     Ashiyaan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [73]:
# function to remove stopwords
def remove_stopwords(text):
    return [word for word in text if word not in stop_words]

In [74]:
train['tokens1'] = train['tokens1'].apply(remove_stopwords)
train['tokens2'] = train['tokens2'].apply(remove_stopwords)

train.head()

,id,qid1,qid2,question1,question2,is_duplicate,tokens1,tokens2
298900,298900,132471,44677,What are some lesser known facts about PM Nare...,What are some lesser known facts/incidents rel...,1,"[What, lesser, known, facts, PM, Narendra, Mod...","[What, lesser, known, facts/incidents, related..."
101435,101435,168039,168040,How do I use PC's internet connection on Andro...,Can I use net on my Android phone from cable b...,0,"[How, I, use, PC, 's, internet, connection, An...","[Can, I, use, net, Android, phone, cable, broa..."
349253,349253,226755,5914,How do you delete your Yahoo email account?,How can you delete your Yahoo mail account?,1,"[How, delete, Yahoo, email, account, ?]","[How, delete, Yahoo, mail, account, ?]"
68381,68381,118184,118185,What is the real world?,What is out there in the real world?,0,"[What, real, world, ?]","[What, real, world, ?]"
102409,102409,169441,169442,Why do artists/musicians/actors etc tend to be...,What is the future of left wing politics in In...,0,"[Why, artists/musicians/actors, etc, tend, lef...","[What, future, left, wing, politics, India, ?]"


#### Removing punctuation

In [75]:
# removing punctuations
import string

In [76]:
def remove_punctuations(text):
    return [word for word in text if word not in string.punctuation]

In [77]:
train['tokens1'] = train['tokens1'].apply(remove_punctuations)
train['tokens2'] = train['tokens2'].apply(remove_punctuations)

#### Normalizing

In [78]:
# normalization
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to C:\Users\Verma
[nltk_data]     Ashiyaan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [79]:
def word_lemmatizer(text):
    lem_text = [lemmatizer.lemmatize(i) for i in text]
    return lem_text

In [80]:
train['tokens1'] = train['tokens1'].apply(word_lemmatizer)
train['tokens2'] = train['tokens2'].apply(word_lemmatizer)

#### Stemming

In [81]:
# stemming
from nltk.stem import PorterStemmer

In [82]:
stemmer = PorterStemmer()

def word_stemmer(text):
    stem_text = " ".join([stemmer.stem(i) for i in text])
    return stem_text

In [83]:
train['tokens1'] = train['tokens1'].apply(word_stemmer)
train['tokens2'] = train['tokens2'].apply(word_stemmer)

train.head()

,id,qid1,qid2,question1,question2,is_duplicate,tokens1,tokens2
298900,298900,132471,44677,What are some lesser known facts about PM Nare...,What are some lesser known facts/incidents rel...,1,what lesser known fact pm narendra modi,what lesser known facts/incid relat narendra modi
101435,101435,168039,168040,How do I use PC's internet connection on Andro...,Can I use net on my Android phone from cable b...,0,how i use pc 's internet connect android phone...,can i use net android phone cabl broadband con...
349253,349253,226755,5914,How do you delete your Yahoo email account?,How can you delete your Yahoo mail account?,1,how delet yahoo email account,how delet yahoo mail account
68381,68381,118184,118185,What is the real world?,What is out there in the real world?,0,what real world,what real world
102409,102409,169441,169442,Why do artists/musicians/actors etc tend to be...,What is the future of left wing politics in In...,0,whi artists/musicians/actor etc tend left wing...,what futur left wing polit india


### Feature Engineering

- tf-idf
- word2vec
- word count
- number of the same words in both questions
- ....

### Modeling

Different modeling techniques can be used:

- logistic regression
- XGBoost
- LSTMs
- etc